In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/migros-datathon-outputs/transaction_sale_with_category_number.csv
/kaggle/input/migros-datathon-outputs/transaction_header.csv
/kaggle/input/migros-datathon-outputs/new_train.csv
/kaggle/input/migros-datathon-outputs/new_test.csv
/kaggle/input/migros-datathon-coderspace/customeraccount.csv
/kaggle/input/migros-datathon-coderspace/sample_submission.csv
/kaggle/input/migros-datathon-coderspace/transaction_header.csv
/kaggle/input/migros-datathon-coderspace/genel_kategoriler.csv
/kaggle/input/migros-datathon-coderspace/customer.csv
/kaggle/input/migros-datathon-coderspace/train.csv
/kaggle/input/migros-datathon-coderspace/test.csv
/kaggle/input/migros-datathon-coderspace/product_groups.csv
/kaggle/input/migros-datathon-coderspace/transaction_sale/transaction_sale.csv


In [24]:
# BU KISIMLA İŞİM KALMADI.
def cardNumberToIndividualNumber(cardnumber):
    customer_info = customeraccount.loc[customeraccount["cardnumber"] == cardnumber]
    return customer_info["individualnumber"].values[0]
def cardNumberToIndividualNumberDf(df): #transaction_header için
    for cardnumber in df["cardnumber"].unique():
        df["cardnumber"] = df["cardnumber"].replace(cardnumber,cardNumberToIndividualNumber(cardnumber))
    df.rename(columns  = {"cardnumber":"individualnumber"},inplace = True)
    return df

In [25]:
# Bu kısımla işim kalmadı.
def categoryLeveltoCategoryNumber():
    transaction_sale_with_category_number = pd.DataFrame()
    for i in range(product_groups.shape[0]):
        values = product_groups.iloc[i]
        temp = transaction_sale.loc[transaction_sale["category_level_1"] == values.category_level_1]
        temp2 = temp.loc[temp["category_level_2"] == values.category_level_2]  
        temp3 = temp2.loc[temp2["category_level_3"] == values.category_level_3]
        temp4 = temp3.loc[temp3["category_level_4"] == values.category_level_4]
        temp4["category_number"] = values.category_number
        if temp4.shape[0] == 0:
            continue
        transaction_sale_with_category_number = transaction_sale_with_category_number.append(temp4)
#         print(transaction_sale_with_category_number.shape[0])
    transaction_sale_with_category_number = transaction_sale_with_category_number[["basketid","category_number","category_level_1","category_level_2","category_level_3","category_level_4","amount","quantity","discount_type_1","discount_type_2","discount_type_3"]]
    return transaction_sale_with_category_number

In [3]:
train = pd.read_csv("/kaggle/input/migros-datathon-coderspace/train.csv")
test = pd.read_csv("/kaggle/input/migros-datathon-coderspace/test.csv")
customeraccount = pd.read_csv("/kaggle/input/migros-datathon-coderspace/customeraccount.csv") # cardnumber-individualnumber eşleşmesi
transaction_header = pd.read_csv("/kaggle/input/migros-datathon-coderspace/transaction_header.csv") # fiş bilgileri
genel_kategoriler = pd.read_csv("/kaggle/input/migros-datathon-coderspace/genel_kategoriler.csv") # Genel kategori isimleri
customer = pd.read_csv("/kaggle/input/migros-datathon-coderspace/customer.csv") # Müşterilerin demografik bilgileri
product_groups = pd.read_csv("/kaggle/input/migros-datathon-coderspace/product_groups.csv") # Kampanya kategorilerinin denk geldiği kategori kırılımları
transaction_sale = pd.read_csv("/kaggle/input/migros-datathon-coderspace/transaction_sale/transaction_sale.csv") # Fişlerin içerisindeki ürünlerin bilgileri

28593 customer

In [4]:
new_transaction_header = pd.read_csv("/kaggle/input/migros-datathon-outputs/transaction_header.csv")
new_transaction_header

,date_of_transaction,individualnumber,basketid,is_sanal
0,2020-12-01,68318409,20120163990041163,0
1,2020-12-01,93939734,20120163990016137,0
2,2020-12-01,58513765,20120160080017338,0
3,2020-12-01,93345438,20120163570105022,0
4,2020-12-01,98388774,20120162340016645,0
...,...,...,...,...
1124668,2021-12-01,93750184,21120111950019800,0
1124669,2021-12-01,95229272,21120111830079927,0
1124670,2021-12-01,96005271,21120110690047978,0
1124671,2021-12-01,22456192,21120110000068135,0


In [38]:
liste = new_transaction_header["is_sanal"].tolist()
liste.count(1)

100278

In [37]:
new_transaction_header.loc[new_transaction_header["individualnumber"] == 68318409].is_sanal.unique()

array([0])

ÜRÜNLERDEN KATEGORİYE GEÇMEM ŞART

Önce category_number üzerinden daha sonra category_leveller üzerinden ilerle.

Basketid üzerinden category_number'a ulaşmam gerek.

In [5]:
transaction_sale_with_category_number = pd.read_csv("/kaggle/input/migros-datathon-outputs/transaction_sale_with_category_number.csv")
transaction_sale_with_category_number

,basketid,category_number,category_level_1,category_level_2,category_level_3,category_level_4,amount,quantity,discount_type_1,discount_type_2,discount_type_3
0,20121105660035926,9046,341,20,10,10,35.90,2.0,3.370627,0.0,0.0
1,20122502190039251,9046,341,20,10,10,11.65,1.0,NaN,0.0,0.0
2,20123102430013537,9046,341,20,10,10,13.95,1.0,0.391479,0.0,0.0
3,20121503940050445,9046,341,20,10,10,23.30,2.0,NaN,0.0,0.0
4,20122506070306537,9046,341,20,10,10,23.30,2.0,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
5434548,21073156480034930,9020,616,95,20,35,21.75,1.0,0.303495,0.0,0.0
5434549,21032363550035024,9020,616,95,20,35,21.75,1.0,3.146503,0.0,0.0
5434550,21041332380033209,9020,616,95,20,35,9.95,1.0,NaN,0.0,0.0
5434551,21042830750024488,9020,616,95,20,35,9.95,1.0,1.539624,0.0,0.0


In [39]:
def controlerIsSanal(liste):
    if not 1 in liste:
        return 0
    if not 0 in liste:
        return 1
    numberof1 = liste.count(1)
    value = numberof1 / len(liste)
    if(value < 0.5):
        return 0
    elif(value >= 0.5):
        return 1

In [57]:
train = pd.read_csv("/kaggle/input/migros-datathon-outputs/new_train.csv")
train

,individualnumber,category_number,hakkedis_amt,odul_amt,response,amount_mean,quantity
0,94230288,9000,21.0,2.0,0,5.956176,1.058824
1,4684087,9000,17.0,1.0,0,9.090909,1.090909
2,92472145,9058,24.0,3.0,0,10.535000,1.100000
3,88026681,9030,22.0,2.0,0,15.610000,1.600000
4,98127795,9001,38.0,3.0,0,19.000000,1.000000
...,...,...,...,...,...,...,...
13110,97214433,9059,27.0,2.0,0,11.792000,1.000000
13111,96841665,9004,65.0,6.0,0,44.712500,1.250000
13112,98445787,9044,33.0,3.0,0,33.240000,0.832000
13113,13781030,9004,66.0,6.0,0,68.500000,1.000000


In [59]:
def createNewTrain():
    new_train = train.copy()
    new_train["gender"] = None
    new_train["dateofBirth"] = None
    new_train["is_sanal"] = None
    for i in range(new_train.shape[0]):
        row = new_train.iloc[i]
        is_sanal_list = new_transaction_header.loc[new_transaction_header["individualnumber"] == row.individualnumber].is_sanal.tolist()
        is_sanal_value = controlerIsSanal(is_sanal_list) 
        gender = customer.loc[customer["individualnumber"] == row.individualnumber].gender
        date_of_birth = customer.loc[customer["individualnumber"] == row.individualnumber].dateofbirth
        new_train.loc[i,"gender"] = gender.values[0]
        new_train.loc[i,"dateofBirth"] = date_of_birth.values[0]
        new_train.loc[i,"is_sanal"] = is_sanal_value
        if i %1000 == 0:
            print(i)
    return new_train

In [60]:
new_train = createNewTrain()
new_train

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000


,individualnumber,category_number,hakkedis_amt,odul_amt,response,amount_mean,quantity,gender,dateofBirth,is_sanal
0,94230288,9000,21.0,2.0,0,5.956176,1.058824,E,1983.0,0
1,4684087,9000,17.0,1.0,0,9.090909,1.090909,E,1951.0,0
2,92472145,9058,24.0,3.0,0,10.535000,1.100000,K,1982.0,0
3,88026681,9030,22.0,2.0,0,15.610000,1.600000,K,1961.0,1
4,98127795,9001,38.0,3.0,0,19.000000,1.000000,E,1984.0,0
...,...,...,...,...,...,...,...,...,...,...
13110,97214433,9059,27.0,2.0,0,11.792000,1.000000,K,1997.0,0
13111,96841665,9004,65.0,6.0,0,44.712500,1.250000,E,1971.0,0
13112,98445787,9044,33.0,3.0,0,33.240000,0.832000,K,1975.0,0
13113,13781030,9004,66.0,6.0,0,68.500000,1.000000,E,1981.0,0


In [68]:
def label_encoder(data):
    data = pd.DataFrame(data = data)
    from sklearn import preprocessing
    le = preprocessing.LabelEncoder()
    columns = data.columns
    for column in columns:
        data[column] = le.fit_transform(data[column])
    return data

In [70]:
new_train["gender"] = label_encoder(new_train["gender"])
new_train

,individualnumber,category_number,hakkedis_amt,odul_amt,response,amount_mean,quantity,gender,dateofBirth,is_sanal
0,94230288,9000,21.0,2.0,0,5.956176,1.058824,0,1983.0,0
1,4684087,9000,17.0,1.0,0,9.090909,1.090909,0,1951.0,0
2,92472145,9058,24.0,3.0,0,10.535000,1.100000,1,1982.0,0
3,88026681,9030,22.0,2.0,0,15.610000,1.600000,1,1961.0,1
4,98127795,9001,38.0,3.0,0,19.000000,1.000000,0,1984.0,0
...,...,...,...,...,...,...,...,...,...,...
13110,97214433,9059,27.0,2.0,0,11.792000,1.000000,1,1997.0,0
13111,96841665,9004,65.0,6.0,0,44.712500,1.250000,0,1971.0,0
13112,98445787,9044,33.0,3.0,0,33.240000,0.832000,1,1975.0,0
13113,13781030,9004,66.0,6.0,0,68.500000,1.000000,0,1981.0,0


In [73]:
test = pd.read_csv("/kaggle/input/migros-datathon-outputs/new_test.csv")
test

,individualnumber,category_number,hakkedis_amt,odul_amt,amount_mean,quantity
0,95812936,9029,38.0,3.0,NaN,NaN
1,23929570,9012,28.0,2.0,NaN,NaN
2,95948115,9019,21.0,2.0,NaN,NaN
3,100898513,9035,48.0,4.0,9.706250,1.840000
4,12769156,9015,26.0,3.0,13.700000,1.750000
...,...,...,...,...,...,...
13231,93418529,9049,32.0,3.0,10.448571,1.571429
13232,97397069,9018,20.0,2.0,6.850000,1.000000
13233,95212479,9046,85.0,8.0,NaN,NaN
13234,93406449,9032,39.0,3.0,29.003333,2.000000


In [76]:
def createNewTest():
    new_test = test.copy()
    new_test["gender"] = None
    new_test["dateofBirth"] = None
    new_test["is_sanal"] = None
    for i in range(new_test.shape[0]):
        row = new_test.iloc[i]
        is_sanal_list = new_transaction_header.loc[new_transaction_header["individualnumber"] == row.individualnumber].is_sanal.tolist()
        is_sanal_value = controlerIsSanal(is_sanal_list) 
        gender = customer.loc[customer["individualnumber"] == row.individualnumber].gender
        date_of_birth = customer.loc[customer["individualnumber"] == row.individualnumber].dateofbirth
        new_test.loc[i,"gender"] = gender.values[0]
        new_test.loc[i,"dateofBirth"] = date_of_birth.values[0]
        new_test.loc[i,"is_sanal"] = is_sanal_value
        if i %1000 == 0:
            print(i)
    return new_test

In [77]:
new_test = createNewTest()
new_test["gender"] = label_encoder(new_test["gender"])
new_test

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000


,individualnumber,category_number,hakkedis_amt,odul_amt,amount_mean,quantity,gender,dateofBirth,is_sanal
0,95812936,9029,38.0,3.0,NaN,NaN,0,1995.0,0
1,23929570,9012,28.0,2.0,NaN,NaN,0,1974.0,0
2,95948115,9019,21.0,2.0,NaN,NaN,0,1967.0,0
3,100898513,9035,48.0,4.0,9.706250,1.840000,1,1978.0,0
4,12769156,9015,26.0,3.0,13.700000,1.750000,1,1970.0,0
...,...,...,...,...,...,...,...,...,...
13231,93418529,9049,32.0,3.0,10.448571,1.571429,0,1979.0,0
13232,97397069,9018,20.0,2.0,6.850000,1.000000,0,1993.0,0
13233,95212479,9046,85.0,8.0,NaN,NaN,0,1962.0,0
13234,93406449,9032,39.0,3.0,29.003333,2.000000,0,1966.0,0


In [80]:
X = new_train[["category_number","hakkedis_amt","odul_amt","amount_mean","quantity","gender","dateofBirth","is_sanal"]].values
Y = new_train[["response"]].values

In [81]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [94]:
new_test

,individualnumber,category_number,hakkedis_amt,odul_amt,amount_mean,quantity,gender,dateofBirth,is_sanal
0,95812936,9029,38.0,3.0,NaN,NaN,0,1995.0,0
1,23929570,9012,28.0,2.0,NaN,NaN,0,1974.0,0
2,95948115,9019,21.0,2.0,NaN,NaN,0,1967.0,0
3,100898513,9035,48.0,4.0,9.706250,1.840000,1,1978.0,0
4,12769156,9015,26.0,3.0,13.700000,1.750000,1,1970.0,0
...,...,...,...,...,...,...,...,...,...
13231,93418529,9049,32.0,3.0,10.448571,1.571429,0,1979.0,0
13232,97397069,9018,20.0,2.0,6.850000,1.000000,0,1993.0,0
13233,95212479,9046,85.0,8.0,NaN,NaN,0,1962.0,0
13234,93406449,9032,39.0,3.0,29.003333,2.000000,0,1966.0,0


In [95]:
new_train.to_csv("new_train1312.csv",index = False)
new_test.to_csv("new_test1312.csv",index = False)

In [89]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
model_xgb = xgb.fit(X_train,y_train)
y_pred = model_xgb.predict(X_test)
# import statsmodels.api as sm
# model = sm.OLS(y_pred,X).fit() 
# print(model.summary())  
# print(f"XGBRegressor, mean_squared_error = {sqrt(mean_squared_error(y_pred, Y))}") 

In [90]:
# new_test["individualnumber"] = new_test["individualnumber"].astype(str)
X = new_test[["category_number","hakkedis_amt","odul_amt","amount_mean","quantity","gender","dateofBirth","is_sanal"]].values
y_pred = model_xgb.predict(X)
# Y = new_test[["response"]].values

In [84]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [91]:
submission = pd.DataFrame()
submission["individualnumber"] = new_test["individualnumber"]
submission["response"] = y_pred
submission = submission.set_index("individualnumber")
submission

,response
individualnumber,
95812936,0
23929570,0
95948115,0
100898513,0
12769156,0
...,...
93418529,0
97397069,0
95212479,0


In [92]:
submission.to_csv("predictions1312.csv")

In [7]:
def gettingInvidualNumberWithBasketId(basketid):
    row = new_transaction_header.loc[new_transaction_header["basketid"] == basketid]
    return row["individualnumber"].values[0]

def addingIndiviualNumberWithBasketId(df): #transaction_sale için
    new_transaction_sale = pd.DataFrame()
    for basketid in df["basketid"].unique(): # 1 mily
        individualnumber = gettingInvidualNumberWithBasketId(basketid)
        temp = df.loc[df["basketid"] == basketid]
        temp["individualnumber"] = individualnumber
        new_transaction_sale = new_transaction_sale.append(temp)
    return new_transaction_sale

In [ ]:
def categoryNumberTogenelKategori(category_number):
    kategori_info = genel_kategoriler.loc[genel_kategoriler["category_number"] == category_number]
    return kategori_info["genel_kategori"].values[0]

In [55]:
transaction_header # bu dosyadaki cardnumber'ı müşteri id'si ile değiştireceğiz, daha sonra basketid-müşteri ilişkisi kurulacak.

,date_of_transaction,cardnumber,basketid,is_sanal
0,2020-12-01,8487374630565615,20120163990041163,0
1,2020-12-01,4287574603569015,20120163990016137,0
2,2020-12-01,6487154648560315,20120160080017338,0
3,2020-12-01,3787564693563915,20120163570105022,0
4,2020-12-01,9087994683569715,20120162340016645,0
...,...,...,...,...
1124668,2021-12-01,5187224618568515,21120111950019800,0
1124669,2021-12-01,87714689568015,21120111830079927,0
1124670,2021-12-01,3287454606560415,21120110690047978,0
1124671,2021-12-01,9887854616568815,21120110000068135,0


In [ ]:
def individualnumberToGenderDate(individualnumber):
    customer_info = customer.loc[customer["individualnumber"] == individualnumber]
    gender = customer_info["gender"].values[0]
    dateofBirth = customer_info["dateofbirth"].values[0]
    return [gender,dateofBirth]

In [ ]:
def individualnumberToGenderDatedDf(df): #invidialnumber'ı silme kısmını eklemedim henüz.
    temp_df = df.copy()
    temp_df["gender"] = None
    temp_df["dateofBirth"] = None
    for i in range(0,temp_df.shape[0]):
        row = temp_df.iloc[[i]]
        individualnumber = row["individualnumber"].values[0]
        liste = individualnumberToGenderDate(individualnumber)
        temp_df.loc[i,"gender"] = liste[0]
        temp_df.loc[i,"dateofBirth"] = liste[1]
    return temp_df

In [ ]:
new_train = individualnumberToGenderDatedDf(train)
new_train